In [103]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 12 12:53:44 2019

@author: sunym
"""

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pickle
import pandas as pd

X = pd.read_csv("heart.csv")


def validation(X,Y, TestSize = 0.2,epo = 60):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    X_trainX = X_train[:,7]
    X_trainZ = np.delete(X_train,7,1)
    X_trainX = np.expand_dims(X_trainX,axis=1)

    X_testX = X_test[:,7]
    X_testZ = np.delete(X_test,7,1)
    X_testX = np.expand_dims(X_testX,axis=1)


    #build the model based nn
    Pain = tf.keras.layers.Input(shape = (1,), dtype=tf.float32)
    WOPain = tf.keras.layers.Input(shape = (14,), dtype = tf.float32)

    AlphaZ = tf.keras.layers.Dense(200,activation = 'relu',name = 'AlphaZ_1')(WOPain)
    AlphaZ = tf.keras.layers.Dropout(rate = 0.5,name = 'AlphaZ_Dropout_1')(AlphaZ)
    AlphaZ = tf.keras.layers.Dense(10,activation = 'relu',name = 'AlphaZ_2')(AlphaZ)
    AlphaZ = tf.keras.layers.Dropout(rate = 0.3,name = 'AlphaZ_Dropout_2')(AlphaZ)
    A = tf.keras.layers.Dense(1, activation = 'linear',name = 'AlphaZ_3')(AlphaZ)

    BetaZ = tf.keras.layers.Dense(100,activation = 'relu',name = 'BetaZ_1')(WOPain)
    BetaZ = tf.keras.layers.Dropout(rate = 0,name = 'BetaZ_Dropout_1')(BetaZ)
    BetaZ = tf.keras.layers.Dense(90,activation = 'relu',name = 'BetaZ_2')(BetaZ)
    BetaZ = tf.keras.layers.Dropout(rate = 0,name = 'BetaZ_Dropout_2')(BetaZ)
    BetaZ = tf.keras.layers.Dense(1, activation = 'linear', name = 'BetaZ_3')(BetaZ)


    B = tf.keras.layers.Multiply(name = 'BetaF')([BetaZ,Pain])

    y_pred = tf.keras.layers.Add(name = 'model')([A,B])
    y_pred = tf.keras.layers.Activation('sigmoid')(y_pred)


    LRmodel = tf.keras.models.Model(inputs=[Pain,WOPain], outputs=y_pred)
    LRmodel.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0014),
        metrics=["accuracy"],
    )

    history = LRmodel.fit([X_trainX,X_trainZ], Y_train,epochs=epo,
                validation_data=([X_testX,X_testZ],Y_test),batch_size = 64,verbose = 1)

    _, acc_lr = LRmodel.evaluate([X_testX,X_testZ], Y_test,verbose=0)
    pre_lr = LRmodel.predict([X_testX,X_testZ])
    predict1prob_lr = pre_lr[:,0]
    pre_lr = (pre_lr > 0.5).astype(int)
    tnlr, fplr, fnlr, tplr = confusion_matrix(Y_test,pre_lr).ravel()
    sensitivity_lr = tplr/(tplr+fnlr)
    specificity_lr = tnlr/(tnlr+fplr)
    avg_acclr = (sensitivity_lr + specificity_lr)/2
    fpr_lr, tpr_lr, thresholds_lr = metrics.roc_curve(Y_test,predict1prob_lr)
    AUC_lr = metrics.auc(fpr_lr, tpr_lr)

    del LRmodel
    tf.keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()
    return {'Model Based Accuracy':acc_lr, 'Model Based C Stat':AUC_lr,
            'Model Based Sensitivity':sensitivity_lr,
            'Model Based Specificity':specificity_lr,'Model Based Balance Accuracy':avg_acclr}


#X = np.genfromtxt("correctModelSR32_x.csv",delimiter=",",skip_header=1)
#Y = np.genfromtxt("correctModelSR32_y.csv",delimiter=",",skip_header=1)

X['Sex'] = pd.get_dummies(X['Sex'])['F']
X['ATA'] = pd.get_dummies(X['ChestPainType'])['ATA']
X['NAP'] = pd.get_dummies(X['ChestPainType'])['NAP']
X['ChestPainType'] = pd.get_dummies(X['ChestPainType'])['ASY']
X['LVH'] = pd.get_dummies(X['RestingECG'])['LVH']
X['RestingECG'] = pd.get_dummies(X['RestingECG'])['Normal']
X['ExerciseAngina'] = pd.get_dummies(X['ExerciseAngina'])['Y']
X['Flat'] = pd.get_dummies(X['ST_Slope'])['Flat']
X['ST_Slope'] = pd.get_dummies(X['ST_Slope'])['Down']
Y = X['HeartDisease']
X = X.drop(['HeartDisease'],axis = 1)
X = np.array(X)

Times = 25
ModelBasedAcc = np.zeros((Times,))
ModelBasedC = np.zeros((Times,))
ModelBasedSens = np.zeros((Times,))
ModelBasedSpec= np.zeros((Times,))
ModelBasedBalanceAcc = np.zeros((Times,))
for i in range(0,Times):
    Results = validation(X,Y)
    ModelBasedAcc[i] = Results['Model Based Accuracy']
    ModelBasedC[i] = Results['Model Based C Stat']
    ModelBasedSens[i] = Results['Model Based Sensitivity']
    ModelBasedSpec[i]= Results['Model Based Specificity']
    ModelBasedBalanceAcc[i] = Results['Model Based Balance Accuracy']
    print(i)

Results_all = {'Model Based Acc':ModelBasedAcc,'Model Based C Stat': ModelBasedC,
                 'Model Based Sensitivity':ModelBasedSens,'Model Based Specificity':ModelBasedSpec,
                 'Model Based Balance Accuracy':ModelBasedBalanceAcc}
pickle.dump(Results_all,open("INNER_sr32.p","wb"))
Average_Results = {k: np.mean(v) for k, v in Results_all.items()}
Average_Results

Epoch 1/60
12/12 [==============================] - 2s 24ms/step - loss: 433.0946 - accuracy: 0.5722 - val_loss: 257.1711 - val_accuracy: 0.5652
Epoch 2/60
12/12 [==============================] - 0s 7ms/step - loss: 193.3575 - accuracy: 0.5940 - val_loss: 83.7730 - val_accuracy: 0.5978
Epoch 3/60
12/12 [==============================] - 0s 7ms/step - loss: 123.2949 - accuracy: 0.5817 - val_loss: 107.7641 - val_accuracy: 0.5380
Epoch 4/60
12/12 [==============================] - 0s 7ms/step - loss: 101.3705 - accuracy: 0.6512 - val_loss: 119.1703 - val_accuracy: 0.5598
Epoch 5/60
12/12 [==============================] - 0s 7ms/step - loss: 59.1248 - accuracy: 0.6444 - val_loss: 39.6496 - val_accuracy: 0.6902
Epoch 6/60
12/12 [==============================] - 0s 5ms/step - loss: 23.9431 - accuracy: 0.7398 - val_loss: 24.1650 - val_accuracy: 0.7554
Epoch 7/60
12/12 [==============================] - 0s 6ms/step - loss: 33.1266 - accuracy: 0.7589 - val_loss: 102.8617 - val_accuracy: 0.55

{'Model Based Acc': 0.8017391276359558,
 'Model Based C Stat': 0.8509700926354796,
 'Model Based Sensitivity': 0.8157797128757999,
 'Model Based Specificity': 0.784355187257693,
 'Model Based Balance Accuracy': 0.8000674500667465}

0      0
1      1
2      0
3      1
4      0
      ..
913    1
914    1
915    1
916    1
917    0
Name: HeartDisease, Length: 918, dtype: int64

In [100]:
Average_Results = {k: np.mean(v) for k, v in Results_all.items()}
Average_Results

{'Model Based Acc': 0.7902173936367035,
 'Model Based C Stat': 0.8408669411118769,
 'Model Based Sensitivity': 0.7307497481358192,
 'Model Based Specificity': 0.8641349216455342,
 'Model Based Balance Accuracy': 0.7974423348906767}